In [1]:
import tensorflow as tf
import numpy as np
from numpy import linalg as LA
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder(tf.float32, shape=[784, None])
y_true = tf.placeholder(tf.float32, shape=[10, None])
W = tf.Variable(tf.zeros([10,784]))
b = tf.Variable(tf.zeros([10,1]))
y_pred = tf.matmul(W, x) + b
y_pred.get_shape()

TensorShape([Dimension(10), Dimension(None)])

In [4]:
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
# optimzr = tf.train.AdamOptimizer(1e-4).minimize(loss)

# correct_prediction = tf.equal(tf.argmax(y_pred, 0), tf.argmax(y_true, 0))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [34]:
def tf_frobenius_norm(M):
    return tf.reduce_sum(M ** 2) ** 0.5

# to implement nuclear norm
def tf_nuclear_norm(M):
    return 1

def Cgd_Fn(grad, wt):
    return ((1 - alpha ) / alpha) * (wt - lam1 * grad / tf_frobenius_norm(grad))

def Cgd_Nn(grad, wt):
    return ((1 - alpha ) / alpha) * (wt - lam2 * tf_nuclear_norm(grad))


In [49]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

# hyper-parameters
alpha = tf.placeholder_with_default(tf.constant(0.999), tf.constant(0.999).shape)
lam1 = tf.placeholder_with_default(tf.constant(100.0), tf.constant(100.0).shape)
lam2 = tf.placeholder_with_default(tf.constant(100.0), tf.constant(100.0).shape)

# Gradient Descent optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate = alpha)

# Compute the gradients for a list of variables.
grads_and_vars = opt.compute_gradients(loss)

# Normal GD update
gv_sgd = [(gv[0], gv[1]) for gv in grads_and_vars]
optimizer_gv_sgd = opt.apply_gradients(gv_sgd)

# CGD with FN
gv_cgd_fn = [(Cgd_Fn(gv[0], gv[1]), gv[1]) for gv in grads_and_vars]
optimizer_gv_cgd_fn = opt.apply_gradients(gv_cgd_fn)
a = grads_and_vars[0][0]
b = tf_frobenius_norm(grads_and_vars[0][0])
c = grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0])
d = Cgd_Fn(grads_and_vars[0][0], grads_and_vars[0][1])
e = grads_and_vars[0][1]
f = alpha #((1 - alpha ) / alpha)#*(grads_and_vars[0][1] - lam1 * grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0]))


# CGD with NN
gv_cgd_nn = [(Cgd_Nn(gv[0], gv[1]), gv[1]) for gv in grads_and_vars]
optimizer_gv_cgd_Nn = opt.apply_gradients(gv_cgd_nn)

correct_prediction = tf.equal(tf.argmax(y_pred, 0), tf.argmax(y_true, 0))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100000):
        batch = mnist.train.next_batch(100)
        feed_dict = {x: np.transpose(batch[0]), y_true: np.transpose(batch[1])}
        train_accuracy, loss_, g_, W_ = sess.run([accuracy, loss, gv_cgd_fn, W], feed_dict)
        a_,b_,c_,d_,e_,f_ = sess.run([a,b,c,d,e,f], feed_dict)
            
        if i % 100 == 0:
            print('train_accuracy', train_accuracy, loss_)
            print('frob_nrom', LA.norm(a_), b_, LA.norm(c_), LA.norm(g_[0][0]), LA.norm(d_), LA.norm(e_), LA.norm(f_))
        sess.run(optimizer_gv_cgd_fn, feed_dict)
    
    feed_dict={x: np.transpose(mnist.test.images), y_true: np.transpose(mnist.test.labels)}
    test_accuracy = sess.run(accuracy, feed_dict)
    print('test_accuracy', test_accuracy)

train_accuracy 0.07 46.0517
frob_nrom 21.5567 21.5567 1.0 0.100099 0.100099 0.0 0.999
train_accuracy 0.0 147.866
frob_nrom 23.3375 23.3375 1.0 0.0923332 0.0923332 8.71444 0.999
train_accuracy 0.0 285.628
frob_nrom 23.6118 23.6118 1.0 0.0851485 0.0851485 16.5792 0.999
train_accuracy 0.0 363.602
frob_nrom 21.5782 21.5782 1.0 0.0787868 0.0787868 23.7374 0.999
train_accuracy 0.0 520.301
frob_nrom 23.9385 23.9385 1.0 0.0736236 0.0736236 30.2167 0.999
train_accuracy 0.01 624.777
frob_nrom 23.4221 23.4221 1.0 0.0692606 0.0692606 36.0751 0.999
train_accuracy 0.01 653.222
frob_nrom 21.6787 21.6787 1.0 0.0643617 0.0643617 41.3617 0.999
train_accuracy 0.0 753.081
frob_nrom 23.3796 23.3796 1.0 0.062449 0.062449 46.1536 0.999
train_accuracy 0.0 818.315
frob_nrom 23.1236 23.1236 1.0 0.0583339 0.0583339 50.4769 0.999
train_accuracy 0.02 820.126
frob_nrom 21.9488 21.9488 1.0 0.0544072 0.0544072 54.3494 0.999
train_accuracy 0.0 902.093
frob_nrom 22.4067 22.4067 1.0 0.0570773 0.0570773 57.8884 0.999
tra

In [ ]:
print(type(grads_and_vars[0][0]))

In [9]:
a = tf.constant([[1,2],[3,4],[5,6]])
b = a**2
tf.Session().run(b)

array([[ 1,  4],
       [ 9, 16],
       [25, 36]], dtype=int32)